In [48]:
import numpy as np

# 输入a矩阵 返回 a = p @ g
def modified_gram_schmid(a):
    # 可能有某一列 是前面的线性组合。
    orth_num = 0
    g = np.full((a.shape[0], a.shape[1]), 0.0)
    p = np.full((a.shape[0], a.shape[1]), 0.0)
    for i in range(a.shape[1]):
        s = a[:, i:i+1]
        # 是前面的线性组合，不能构成正交分量
        if np.linalg.norm(s) == 0:
            continue
        s = s / np.linalg.norm(s)
        p[:, orth_num:orth_num+1] = s
        # 求投影大小
        g[orth_num:orth_num+1, :] =  s.T @ a
        # 减去投影
        a = a - s @ s.T @ a
        orth_num = orth_num + 1
    # 裁剪
    p = p[:a.shape[0], :orth_num]
    g = g[:orth_num, :a.shape[1]]
    return p, g

# 输入a矩阵 返回 a = p.T @ a'
def houshold(a):
    g = np.full((a.shape[0], a.shape[1]), 0.0)
    p = np.eye(a.shape[0])

    # 统一类型，int 和float运算 结果不对 orth_num : 可能某一列全为0
    a, p, g = a.astype(float),p.astype(float), g.astype(float)
    orth_num = 0
    
    for i in range(a.shape[1]):
        s = a[orth_num:, i:i+1]
        # 0不需要 反射变换
        if np.linalg.norm(s) == 0:
            continue
        # 是反射的 也不需要
        s = s/np.linalg.norm(s) - np.eye(a.shape[0] - orth_num)[:, 0:1]
        if np.linalg.norm(s) != 0:
            s = s/ np.linalg.norm(s)
        # 设置反射矩阵
        s = np.eye(a.shape[0] - orth_num) - 2 * s @ s.T
        p[orth_num:, :] = s @ p[orth_num:, :]
        a[orth_num:, i:] = s @ a[orth_num:, i:]
        orth_num = orth_num + 1
        
        if orth_num == a.shape[0]:
            break
    return p.T, a

# 输入a矩阵 返回 a = p.T @ a'
def given(a):
    g = np.full((a.shape[0], a.shape[1]), 0.0)
    p = np.eye(a.shape[0])
    # 统一类型，int 和float运算 结果不对 orth_num : 可能某一列全为0
    a, p, g = a.astype(float),p.astype(float), g.astype(float)
    orth_num = 0
    for i in range(a.shape[1]):
        # 不需要进行旋转变换
        if np.linalg.norm(a[orth_num:, i:]) == 0:
            continue
        # 依次旋转
        for j in range(orth_num+1, a.shape[0]):
            if a[j, i] == 0:
                continue
            s = np.eye(a.shape[0])
            # 求 三条边 然后设定 cos sin角度位置。
            x, y, z  = a[i, i], a[j, i], np.sqrt(a[i, i]**2 + a[j, i]**2)
            s[i, i], s[i, j], s[j, i], s[j, j] = x/z, y/z, y/z, -x/z
            
            p, a = s @ p, s @ a
        orth_num = orth_num + 1
        if orth_num == a.shape[0]:
            break
    return p.T, a


# 生成 测试矩阵
def generate_random_matrix_with_rank_via_svd(m, n, rank):
    # 生成随机矩阵
    A = np.random.rand(m, n)
    
    # 进行 SVD 分解
    U, S, Vt = np.linalg.svd(A, full_matrices=False)
    # print(S)
    # 将奇异值数组中只保留 `rank` 个非零值
    S[rank:] = 0
    S = np.diag(S)
    
    # 重构矩阵
    return U @ S @ Vt

# 输入： A、 b、 mode=1是houshold变换，2是given变换，0是modified gram schmid变换
# 返回： x
def calc(A, b, mode):
    # A = QR
    if mode == 1:
        Q,R = houshold(A)
    elif mode == 2:
        Q,R = given(A)
    else:
        Q,R = modified_gram_schmid(A)
    
    # QRx = b  ==>   Rx = Q^Tb
    b = Q.T @ b
    x = np.full((A.shape[1], 1), 0.0)
    # 去除较小的，要不然算出来的好大。 使用 np.linalg.qr(A)得到的Q,没有这步平方误差也好大。
    # 这可能就是QR分解的不好之处吧。
    R_max = max(np.max(np.abs(R)), np.max(np.abs(b))) * 1e-5
    R[np.abs(R) < R_max] = 0
    b[np.abs(b) < R_max] = 0

    # 求 主元所在行和列
    tmp = 0
    list = []
    for i in range(R.shape[1]):
        if R[tmp, i] != 0:
            list.append((tmp, i))
            tmp += 1
            if tmp == R.shape[0]:
                break
    list.reverse()
    
    # 解方程
    for id, (i, j) in enumerate(list):
        x[j, 0] = b[i, 0] / R[i, j]
        for k in range(0, i):
            b[k, 0] = b[k, 0] - R[k, j] * x[j, 0]

    return x

def pri(a, x, b):
    ax = a @ x
    ax_b = ax-b
    dis = np.sum((a@x-b)**2)

    table = np.column_stack((x, ax, b, ax_b))
    for row in table:
        print('\t'.join(f"{value:.10f}" for value in row))
    print("平方误差和: ",dis)
    # print(f"x    : {x.T} \nAx   : {(a@x).T} \n误差 : {np.sum((a@x-b)**2)}\n\n")

a = np.full((5, 5), 1.0)
a = generate_random_matrix_with_rank_via_svd(5, 5, 3)
# a = np.array([[1, 19, -34], [-2, -5, 20], [2, 8, 37]])

b = np.full((a.shape[0], 1), 1)


x = calc(a, b, 0)
print("A矩阵: ")
for row in a:
    print('\t'.join(f"{value: .10f}" for value in row))
print("b矩阵: ")
for row in b:
    print('\t'.join(f"{value: .10f}" for value in row))
print("my_solve : \n")
pri(a, x, b)
x = np.linalg.pinv(a) @ b

print("\nnp.linalg.pinv_solve : \n")
pri(a, x, b)


A矩阵: 
 0.8576527711	 0.4701763811	-0.0410796381	 0.1590006008	 0.2810071397
 0.7820326753	 0.6060340036	 0.1824042956	 0.2908989028	 0.2427210122
 0.3147815233	 0.8785895619	 0.9779334030	 0.9881670919	 0.4525131865
 0.7641257111	 0.9009491992	 0.7077013792	 0.9732261911	 0.7162938501
 0.7077948197	 0.3680422376	 0.1053573051	 0.6014179870	 0.7959253855
b矩阵: 
 1.0000000000
 1.0000000000
 1.0000000000
 1.0000000000
 1.0000000000
my_solve : 

1.3538630742	0.9543821769	1.0000000000	-0.0456178231
-0.3732501433	0.9714046739	1.0000000000	-0.0285953261
0.7611761020	0.8426169367	1.0000000000	-0.1573830633
0.0000000000	1.2369275438	1.0000000000	0.2369275438
0.0000000000	0.9010809154	1.0000000000	-0.0989190846
平方误差和:  0.09358775336205041

np.linalg.pinv_solve : 

0.7811492373	0.9543821769	1.0000000000	-0.0456178231
0.4916249372	0.9714046739	1.0000000000	-0.0285953261
0.0075348926	0.8426169367	1.0000000000	-0.1573830633
0.0971539113	1.2369275438	1.0000000000	0.2369275438
0.1357223001	0.9010809154